# SQLITE3  in Python

SQLlite is a package to manipulate dataset using the Structured Query Language.

In this notebook we introduce the concept of a SQL database and show how to interact with these databases in Python.

The fundamental concepts in SQL are the following:
* a database is a collection of datasets stored as tables.
  * Tables are similar to a pandas dataframe but the index is called rowid. 
  * We use SQL commands to create and drop tables in the database
  * CREATE allows us to create a new table
  * DROP allows us to delete an existing table and all of its data
* a table is a dataset consisting of a number of rows and columns
  *  each column has a name and a type
  *  each row has a unique rowid automatically created by SQLite
* a cursor is a Python object that allows us to interact with the tables in a database in several ways
  * INSERT allows us to add new rows
  * UPDATE allows us to modify the content of a row
  * DELETE allows us to remove rows
  * SELECT allows us to return rows of data from the tables in the database

Here is a link to all of the [SQLite statements](https://www.sqlite.org/lang.html) (and their grammar)

Here is a page 
[Pandas vs SQL](https://pandas.pydata.org/docs/getting_started/comparison/comparison_with_sql.html)
that compares Pandas with SQL commands since they both deal with tables of data and do similar things.

As usual we first import the libraries we will need.


In [1]:
import sqlite3
import csv
'done'

'done'

# Interacting with the database
Sqlite stores its data in a file and to interact with the database you need to take three steps:
1. create a connection to the database (and this will create the database file if it doesn't already exist)
1. create a cursor for the connection (this allows us to query and modify the database)
1. run some queries using the cursor
1. commit the changes (if we don't commit the change then the database isn't updated, it is similar to commit with git!)
1. close the connection

The template for this code looks like this
``` python
    con= sqlite3.connect('courses.db') # create the connection
    cur = con.cursor()   # create the cursor to interact with the connection
    cur.execute(''' SOME 
    (MULTILINE) SQL COMMANDS GO HERE
    ending with a semicolon, always  ;
    ''')
    con.commit()  # write any database changes to the database file
    con.close()  # close the connection
```

In this tutorial we will learn a few of the core SQL commands. 
The full list is here https://www.sqlite.org/lang.html

# Overview of most common SQL commands
## Creating and dropping tables
``` python
CREATE TABLE tablename IF NOT EXISTS (colnam1 coltype1,  colname2 coltyp2, ...); -- create a new table
DROP TABLE tablename;
```
## Inserting data into a table
``` python
INSERT INTO tablename VALUES (col1val, col2val, ...) ; -- insert a row into a table
```
## Selecting data from a table
``` python
SELECT * FROM tablename;
SELECT colname1, colname2, ... FROM tablename WHERE condition1 OR condition2 AND NOT condition3;
```
## Delete rows from the table
``` python
DELETE FROM tablename;  -- remove all rows from the table
DELETE FROM tablename WHERE conditions;  -- delete selected rows from the table
```
## Updating data 
``` python
UPDATE tablename SET colname=expression WHERE condition;
```

# CRUD
There are many more SQL commands but these five will allow you to do all of the CRUD operations
which form the foundation of most database-backed apps. If you are interested in learning more about
databases you should take the **CS127b: Introduction to Databases** course
* Create -- with INSERT
* Read -- with SELECT
* Update -- with UPDATE
* Delete -- with DELETE

In the rest of this notebook we show how to interact with a SQLite database using Python.

We will use the courses.csv dataset as our example.

# Creating a table to store the course data
SQL requires that all data be stored in tables where each column has a name and a type.

The SQLite types are TEXT, NUMERIC, INTEGER, REAL, BLOB 

Here we define a function to create the table we are going to use to store the PA01 dataset.

The PA01 dataset, courses.csv, has the following form as a CSV file:
```
subj	num	suf	format	term	code	inst	title	sec	enr
NEJS	188	A	LEC	Fall 2004	1043	Levy, Avigdor	RISE/DECLINE:OTTOMAN EMP	1	32
CHEM	121	A	LEC	Fall 2004	1043	Foxman, Bruce	INORGANIC CHEM I, LECS.	1	18
CHEM	130	A	LEC	Fall 2004	1043	Yu, Jinquan	ADV ORG. CHEM: STRUCTURE	1	14
```
The functions below create or drop the table to store the courses.csv data.

In [2]:
def create_data_table():
    ''' create a table to store the Brandeis course data'''
    con= sqlite3.connect('courses.db')
    cur = con.cursor()
    cur.execute('''CREATE TABLE IF NOT EXISTS data
                     (subj text, num int, suf text, format text, term text, code int, inst text, title text, sec text, enr int)''')
    con.commit()
    con.close()

def drop_data_table():
    ''' remove the table and all of its data from the database'''
    con= sqlite3.connect('courses.db')
    cur = con.cursor()
    cur.execute('''DROP TABLE IF EXISTS data ''')
    con.commit()
    con.close()
    
drop_data_table()
create_data_table()

# loading and selecting the data
To load data into a database we can use the executemany cursor command
with an "INSERT SQL" command. Each row in rows is a tuple with 10 elements (matching the 10 question marks in the quer)




In [3]:
def load_data():
    ''' load data from courses.csv into courses.db by inserting one row at a time '''
    con= sqlite3.connect('courses.db')
    cur = con.cursor()
    course_file = open('data/courses.csv')
    rows = csv.reader(course_file)
    for row in rows:
        cur.execute("INSERT INTO data VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)",row)
    con.commit()
    con.close()

def load_data2():
    ''' load data from courses.csv into courses.db using the executemany cursor method '''
    con= sqlite3.connect('courses.db')
    cur = con.cursor()
    course_file = open('data/courses.csv')
    rows = csv.reader(course_file)
    cur.executemany("INSERT INTO data VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)",rows)
    con.commit()
    con.close()
    
def get_all_data():
    ''' read all of the data and return it as a list of tuples '''
    con= sqlite3.connect('courses.db')
    cur = con.cursor()
    results = cur.execute("SELECT * FROM data")
    data= [x for x in results]
    con.commit()
    con.close()
    return data

def remove_all_data():
    ''' remove all of the data from the table '''
    con= sqlite3.connect('courses.db')
    cur = con.cursor()
    results = cur.execute("DELETE FROM data")
    con.commit()
    con.close()

load_data()


# Selecting the data
We can use the select statement to pick rows out of the database and then access those rows with
``` python
cur.fetchone() -- returns the next row in the result
cur.fetchman(N) -- returns the next N rows as a list
cur.fetchall() -- returns all the rows as a list
```

Here are some examples..

In [4]:
con= sqlite3.connect('courses.db')
cur = con.cursor()


print("First 10 courses in the dataset")
cur.execute("SELECT * FROM data;")
for i in range(10):
    rows = cur.fetchmany(1)
    print(rows)
print('-'*40)


print("Last 10 courses in the dataset")
cur.execute("SELECT * FROM data;")
data = cur.fetchall();
for row in data[-10:]:
    print(row)
print('-'*40)


print("first 100 or fewer COSI courses taught in summer semesters")
cur.execute("SELECT * FROM data where subj='COSI' and code%10=2;")
data = cur.fetchmany(100)
for row in data:
    print(row)
print('-'*40)


con.commit()
con.close()


First 10 courses in the dataset
[('subj', 'num', 'suf', 'format', 'term', 'code', 'inst', 'title', 'sec', 'enr')]
[('NEJS', 188, 'A', 'LEC', 'Fall 2004', 1043, 'Levy, Avigdor', 'RISE/DECLINE:OTTOMAN EMP', '1', 32)]
[('CHEM', 121, 'A', 'LEC', 'Fall 2004', 1043, 'Foxman, Bruce', 'INORGANIC CHEM I, LECS.', '1', 18)]
[('CHEM', 130, 'A', 'LEC', 'Fall 2004', 1043, 'Yu, Jinquan', 'ADV ORG. CHEM: STRUCTURE', '1', 14)]
[('COMP', 1, 'A', 'LEC', 'Fall 2004', 1043, 'Ruesch, Gordon', 'COMPOSITION', '3', 11)]
[('MUS', 118, 'A', 'STD', 'Fall 2004', 1043, 'Ray, Mary', 'SENIOR RECITAL I', '1', 3)]
[('PE', 32, 'A', 'NCA', 'Fall 2004', 1043, 'Dibble, Susan', 'MODERN DANCE', '1', 15)]
[('USEM', 89, 'B', 'SEM', 'Fall 2004', 1043, 'Engerman, David', 'COLLEGE 101', '1W', 18)]
[('HS', 529, 'A', 'LEC', 'Fall 2004', 1043, 'Capitman, John', 'SOC.POLICY/POPULTN GROUP', '1', 15)]
[('POL', 14, 'B', 'LEC', 'Fall 2004', 1043, 'Teles, Steven', "INTRO TO AMERICAN GOV'T.", '1', 81)]
-------------------------------------

# Creating a SQLite shell

In [6]:
def shell():
    
    
    command = input(">> ")
    while command !='quit':
        con= sqlite3.connect('courses.db')
        cur = con.cursor()
        
        cur.execute(command)
        results = cur.fetchall()
        for row in results[:5]:
            print(row)
        print('.'*40)
        for row in results[-5:]:
            print(row)
        con.commit()
        con.close()
        
        command = input(">> ")
    print('bye')
    
    

        
    

In [7]:
shell()

>>  select * from data limit 5;


('subj', 'num', 'suf', 'format', 'term', 'code', 'inst', 'title', 'sec', 'enr')
('NEJS', 188, 'A', 'LEC', 'Fall 2004', 1043, 'Levy, Avigdor', 'RISE/DECLINE:OTTOMAN EMP', '1', 32)
('CHEM', 121, 'A', 'LEC', 'Fall 2004', 1043, 'Foxman, Bruce', 'INORGANIC CHEM I, LECS.', '1', 18)
('CHEM', 130, 'A', 'LEC', 'Fall 2004', 1043, 'Yu, Jinquan', 'ADV ORG. CHEM: STRUCTURE', '1', 14)
('COMP', 1, 'A', 'LEC', 'Fall 2004', 1043, 'Ruesch, Gordon', 'COMPOSITION', '3', 11)
........................................
('subj', 'num', 'suf', 'format', 'term', 'code', 'inst', 'title', 'sec', 'enr')
('NEJS', 188, 'A', 'LEC', 'Fall 2004', 1043, 'Levy, Avigdor', 'RISE/DECLINE:OTTOMAN EMP', '1', 32)
('CHEM', 121, 'A', 'LEC', 'Fall 2004', 1043, 'Foxman, Bruce', 'INORGANIC CHEM I, LECS.', '1', 18)
('CHEM', 130, 'A', 'LEC', 'Fall 2004', 1043, 'Yu, Jinquan', 'ADV ORG. CHEM: STRUCTURE', '1', 14)
('COMP', 1, 'A', 'LEC', 'Fall 2004', 1043, 'Ruesch, Gordon', 'COMPOSITION', '3', 11)


>>  select subj,num,suf from data limit 20;


('subj', 'num', 'suf')
('NEJS', 188, 'A')
('CHEM', 121, 'A')
('CHEM', 130, 'A')
('COMP', 1, 'A')
........................................
('JAPN', 105, 'A')
('USEM', 6, 'A')
('PHYS', 113, 'A')
('YDSH', 30, 'A')
('AAAS', 163, 'B')


>>  quit


bye


In [8]:
drop_data_table()
create_data_table()
data = get_all_data()
print(len(data))

0


In [9]:

load_data()


In [10]:
data = get_all_data()
len(data)

35207

In [11]:
print(data[1000])

('EL', 94, 'A', 'LAB', 'Fall 2009', 1093, 'Morris, James', 'EXPERIENTIAL LEARNING PRACTICM', '8', 1)


In [12]:
print('removing all data from the table')
remove_all_data()
data = get_all_data()
print('tbe table size is now',len(data))
print('.'*40)
print('loading all data into the table')
load_data()
data = get_all_data()
print('the table size is now',len(data))




removing all data from the table
tbe table size is now 0
........................................
loading all data into the table
the table size is now 35207


In [13]:
print('Calculating the unique term codes in the dataset')
con= sqlite3.connect('courses.db')
cur = con.cursor()
cur.execute("SELECT distinct code from data ")
codes = [row[0] for row in cur.fetchall()]
codes = codes[1:]
print(codes)
con.close()

Calculating the unique term codes in the dataset
[1043, 1051, 1052, 1053, 1061, 1062, 1063, 1071, 1072, 1073, 1081, 1082, 1083, 1091, 1092, 1093, 1101, 1102, 1103, 1111, 1112, 1113, 1121, 1122, 1123, 1131, 1132, 1133, 1141, 1142, 1143, 1151, 1152, 1153, 1161, 1162, 1163, 1171, 1172, 1173, 1181, 1182, 1183, 1191, 1192, 1193, 1201, 1202, 1203, 1211, 1212]


In [21]:
print('calculating the number of students taking COSI courses each term (with double counting!)')
con= sqlite3.connect('courses.db')
cur = con.cursor()
#cur.execute("DELETE FROM data WHERE code NOTNULL")
cur.execute("SELECT code,sum(enr) as n ,max(enr) as m,sum(enr)/count(enr) as average from data  where enr>=8 group by code")
for row in cur.fetchall():
    print(row)
con.close()

calculating the number of students taking COSI courses each term (with double counting!)
(1043, 17680, 206, 24)
(1051, 16750, 199, 23)
(1052, 1309, 46, 17)
(1053, 18651, 229, 25)
(1061, 17010, 197, 24)
(1062, 1209, 49, 18)
(1063, 19458, 184, 25)
(1071, 17972, 187, 23)
(1072, 1253, 40, 17)
(1073, 19388, 200, 24)
(1081, 17877, 200, 23)
(1082, 1269, 46, 16)
(1083, 19249, 192, 24)
(1091, 18139, 214, 24)
(1092, 1302, 39, 14)
(1093, 20064, 192, 25)
(1101, 19244, 170, 24)
(1102, 1534, 54, 16)
(1103, 20416, 196, 25)
(1111, 19868, 164, 24)
(1112, 1690, 51, 15)
(1113, 21220, 240, 25)
(1121, 20622, 179, 25)
(1122, 1613, 56, 15)
(1123, 21406, 243, 25)
(1131, 20298, 193, 24)
(1132, 1941, 71, 15)
(1133, 21689, 196, 25)
(1141, 20736, 253, 24)
(1142, 1995, 87, 15)
(1143, 22484, 302, 26)
(1151, 21055, 176, 24)
(1152, 2006, 66, 14)
(1153, 21756, 221, 25)
(1161, 20092, 183, 24)
(1162, 2196, 63, 17)
(1163, 21705, 209, 25)
(1171, 20054, 176, 24)
(1172, 2065, 69, 15)
(1173, 21495, 188, 25)
(1181, 19986, 196

In [16]:
print('Printing out the bulletin of all COSI courses offered from Fall04 to Summer21')
con= sqlite3.connect('courses.db')
cur = con.cursor()
for code in codes[:2]:
    cur.execute("SELECT * from data where subj=(?) and code=(?) order by code, num",('COSI', code ))
    print("Year ",code)
    for row in cur.fetchall():
        print(row)
    print('------',end="\n\n")
con.close()

Printing out the bulletin of all COSI courses offered from Fall04 to Summer21
Year  1043
('COSI', 2, 'A', 'LEC', 'Fall 2004', 1043, 'Hickey, Timothy', 'INTRO TO COMPUTERS', '1', 86)
('COSI', 21, 'A', 'LEC', 'Fall 2004', 1043, 'Storer, James', 'STRUCTURE/FUNDMT.COMPUT.', '1', 23)
('COSI', 22, 'A', 'LAB', 'Fall 2004', 1043, 'Storer, James', 'FUNDAMENTALS:PROGRAMMING', '1', 23)
('COSI', 29, 'A', 'LEC', 'Fall 2004', 1043, 'Gessel, Ira', 'DISCRETE STRUCTURES', '1', 27)
('COSI', 31, 'A', 'LEC', 'Fall 2004', 1043, 'Cherniack, Mitch', 'COMPUTER STRUCTURES/ORG.', '1', 34)
('COSI', 112, 'A', 'LEC', 'Fall 2004', 1043, 'Pustejovsky, James', 'THRY/MODELS:INTEL.BEHAVR', '1', 23)
('COSI', 120, 'A', 'LEC', 'Fall 2004', 1043, 'Cohen, Jacques', 'TOPICS-COMPUTER SYSTEMS', '1', 15)
('COSI', 171, 'A', 'LEC', 'Fall 2004', 1043, 'Cohn, Martin', 'CRYPTOLOGY', '1', 23)
('COSI', 215, 'A', 'SEM', 'Fall 2004', 1043, 'Alterman, Richard', 'ADV.TOPIC:HMAN-CENT COMP', '1', 10)
('COSI', 217, 'A', 'SEM', 'Fall 2004', 1

In [ ]:
print('printing out the COSI courses from Spring 2005')
con= sqlite3.connect('courses.db')
cur = con.cursor()
cur.execute("SELECT num, suf, title, inst from data where subj=(?) and code=(?)",('COSI', 1051))
for row in cur.fetchall():
    print(row)
con.commit()
con.close()

In [ ]:
print("printing out the number of COSI courses taught by each COSI instructor from Fall04 to Summer21")
con= sqlite3.connect('courses.db')
cur = con.cursor()
cur.execute("SELECT inst,sum(enr) as n from data where subj=(?) group by inst order by n desc",('COSI', ))
for row in cur.fetchall():
    print(row)
con.commit()
con.close()

In [ ]:
print("printing out the enrollments in COSI courses from Fall2004 through Summer2021")
con= sqlite3.connect('courses.db')
cur = con.cursor()
cur.execute("SELECT term,sum(enr) as n from data where subj=(?) group by code order by code",('COSI', ))
for row in cur.fetchall():
    print(row)
    if row[0].startswith('Summer'):
        print()
con.commit()
con.close()

In [22]:
print("printing out rowids which are unique keys for each row")
con= sqlite3.connect('courses.db')
cur = con.cursor()
cur.execute("SELECT rowid,enr,*  from data where subj=(?)  order by enr desc limit 20",('COSI', ))
for row in cur.fetchall():
    print(row)
    if row[2].startswith('Summer'):
        print()
con.commit()
con.close()

printing out rowids which are unique keys for each row
(13773, 169, 'COSI', 11, 'A', 'LEC', 'Fall 2018', 1183, 'Hickey, Timothy', 'PROGRAMMING IN JAVA', '2', 169)
(34816, 166, 'COSI', 164, 'A', 'LEC', 'Fall 2020', 1203, 'Hickey, Timothy J', 'Introduction to 3-D Animation', '1', 166)
(12459, 158, 'COSI', 2, 'A', 'LEC', 'Spring 2017', 1171, 'Hickey, Timothy', 'INTRO TO COMPUTERS', '1', 158)
(13328, 155, 'COSI', 155, 'B', 'LEC', 'Spring 2018', 1181, 'Hickey, Timothy', 'COMPUTER GRAPHICS', '1', 155)
(14631, 155, 'COSI', 10, 'A', 'LEC', 'Fall 2019', 1193, 'Hickey, Timothy', 'INTRO PROBLEM SOLVING PYTHON', '2', 155)
(14630, 154, 'COSI', 29, 'A', 'LEC', 'Fall 2019', 1193, 'Cherniack, Mitch', 'DISCRETE STRUCTURES', '1', 154)
(14192, 153, 'COSI', 164, 'A', 'LEC', 'Spring 2019', 1191, 'Hickey, Timothy', 'INTRO 3-D ANIMATION', '1', 153)
(33649, 150, 'COSI', 10, 'A', 'LEC', 'Spring 2021', 1211, 'Hickey, Timothy J', 'Introduction to Problem Solving in Python', '1', 150)
(14629, 149, 'COSI', 10, 'A'

In [79]:
#print("%-25s %5s %-40s  %3s %5s %5s"%('instructor','id','','class','num','total','avg'))
con= sqlite3.connect('courses.db')
cur = con.cursor()
cur.execute("SELECT inst,title,count(*),sum(enr),group_concat(term),num,suf from data where subj=(?) group by inst,title order by inst asc ",('COSI',))
results = cur.fetchall()
for row in results:
    print("%-25s %4d%s %-40s  %3d %5d %5.1f\n   %s\n"%((row[0],row[5], row[6], row[1], row[2], row[3], row[3]/row[2], row[4])))
con.commit()
con.close()

Abi-Antoun, Marwan          12B ADV. PROGRAMMING TECHNIQUES                 1    18  18.0
   Fall 2019

Abi-Antoun, Marwan         177A SCIENTIFIC DATA PROCESSING                  1    22  22.0
   Fall 2019

Alterman, Richard          215A ADV.TOPIC:COMP SUPPORTED COOP.              3    35  11.7
   Fall 2009,Spring 2014,Spring 2017

Alterman, Richard          215A ADV.TOPIC:HMAN-CENT COMP                    1    10  10.0
   Fall 2004

Alterman, Richard          118A COMP SUPPORT COOPERATIVE LEARN              1    17  17.0
   Fall 2012

Alterman, Richard          111A COMPUTATIONAL COGNIT.SCI                    5    90  18.0
   Fall 2005,Fall 2007,Fall 2010,Spring 2016,Spring 2018

Alterman, Richard          118A COMPUTER-SUPPORTED COOPERATION              6   232  38.7
   Spring 2007,Spring 2009,Spring 2011,Fall 2014,Fall 2016,Fall 2018

Alterman, Richard          118A Computer-Supported Cooperation              1    44  44.0
   Fall 2020

Alterman, Richard          125A HUMAN COMPUT